### Imports

In [96]:
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RXGate, RYGate, RZGate, CXGate
from pennylane.transforms import cancel_inverses, merge_rotations, commute_controlled

### Random Circuit Generator

In [53]:
def make_random_circuit(n, depth=100):
    qc = QuantumCircuit(n)
    for _ in range(depth):
        # one-qubit random rotations
        for qubit in range(n):
            gate_type = random.choice([RXGate, RYGate, RZGate])
            angle = random.random() * 2 * 3.14159
            qc.append(gate_type(angle), [qubit])

        # random CNOTs
        for _ in range(n):
            control = random.randrange(n)
            target = random.randrange(n)
            if control != target:
                qc.append(CXGate(), [control, target])
    return qc

def count_gates(qc):
    return sum(qc.count_ops().values())

### Qiskit to PennyLane Converter

In [85]:
def qiskit_to_pennylane_tape(qc):
    ops = []

    for inst, qargs, _ in qc.data:
        name = inst.name.lower()
        # convert qargs (Qubit objects) to integer wire indices using qc.qubits
        wires = [qc.qubits.index(q) for q in qargs]

        if name == "rx":
            angle = float(inst.params[0])
            ops.append(qml.RX(angle, wires=wires))
        elif name == "ry":
            angle = float(inst.params[0])
            ops.append(qml.RY(angle, wires=wires))
        elif name == "rz":
            angle = float(inst.params[0])
            ops.append(qml.RZ(angle, wires=wires))
        elif name == "cx" or name == "cnot":
            ops.append(qml.CNOT(wires=wires))
        else:
            # this case should never occur for circuits generated via make_random_circuit
            pass

    # build a QuantumTape from the ops
    with qml.tape.QuantumTape() as tape:
        for op in ops:
            qml.apply(op)

    return tape

### PennyLane Optimization Functions

In [101]:
def pl_none(tape):
    new_tape, _ = commute_controlled(tape)
    return new_tape[0]

def pl_cancel(tape):
    new_tape, _ = cancel_inverses(tape)
    return new_tape[0]

def pl_merge(tape):
    new_tape, _ = merge_rotations(tape)
    return new_tape[0]

def pl_both(tape):
    new_tape, _ = qml.compile(tape)
    return new_tape[0]

### Gates vs Qubits Plot

In [56]:
def qiskit_plot_gates_vs_qubits(results):
    plt.figure(figsize=(10, 6))
    for name in results["qiskit_gates"]:
        plt.plot(
            results["qubits"],
            results["qiskit_gates"][name],
            marker='o',
            markersize=4,
            label=f"Optimization Level {name}"
        )
    plt.title("Gate Count vs. Number of Qubits via Qiskit")
    plt.xlabel("Number of Qubits")
    plt.ylabel("Gate Count After Qiskit Transpilation")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def pennylane_plot_gates_vs_qubits(results):
    plt.figure(figsize=(10, 6))
    for name in results["pennylane_gates"]:
        plt.plot(
            results["qubits"],
            results["pennylane_gates"][name],
            marker='o',
            markersize=4,
            label=f"Optimization Method: {name}"
        )
    plt.title("Gate Count vs. Number of Qubits via PennyLane")
    plt.xlabel("Number of Qubits")
    plt.ylabel("Gate Count After PennyLane Optimization")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

### Time vs Qubits Plot

In [57]:
def qiskit_plot_time_vs_qubits(results):
    plt.figure(figsize=(10, 6))
    for name in results["qiskit_times"]:
        plt.plot(
            results["qubits"],
            results["qiskit_times"][name],
            marker='o',
            markersize=4,
            label=f"Optimization Level {name}"
        )
    plt.title("Compilation Time vs. Number of Qubits via Qiskit")
    plt.xlabel("Number of Qubits")
    plt.ylabel("Time (seconds)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def pennylane_plot_time_vs_qubits(results):
    plt.figure(figsize=(10, 6))
    for name in results["pennylane_times"]:
        plt.plot(
            results["qubits"],
            results["pennylane_times"][name],
            marker='o',
            markersize=4,
            label=f"Optimization Method: {name}"
        )
    plt.title("Compilation Time vs. Number of Qubits via PennyLane")
    plt.xlabel("Number of Qubits")
    plt.ylabel("Time (seconds)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

### Main Function

In [ ]:
def main():
    print("Running Qiskit + PennyLane circuit optimization benchmarks")

    simulator = AerSimulator()
    NUM_RUNS = 8

    # data storage
    results = {
        "qubits": [],
        "qiskit_gates": {0: [], 1: [], 2: [], 3: []},
        "qiskit_time":  {0: [], 1: [], 2: [], 3: []},
        "pennylane_gates": {"none": [], "cancel": [], "merge": [], "both": []},
        "pennylane_time":  {"none": [], "cancel": [], "merge": [], "both": []}
    }

    for n in range(2, 31):
        print(f"\n=== Testing n={n} qubits ===")
        results["qubits"].append(n)

        # temporary accumulators
        q_gates = {0: [], 1: [], 2: [], 3: []}
        q_time  = {0: [], 1: [], 2: [], 3: []}
        pl_gates = {"none": [], "cancel": [], "merge": [], "both": []}
        pl_time  = {"none": [], "cancel": [], "merge": [], "both": []}

        for run in range(NUM_RUNS):
            print(f"  Trial {run + 1}/{NUM_RUNS}")

            qc = make_random_circuit(n, depth=500)

            # Qiskit
            for level in range(4):
                start = time.time()
                compiled = transpile(qc, simulator, optimization_level=level)
                elapsed = time.time() - start

                gates = count_gates(compiled)

                q_gates[level].append(gates)
                q_time[level].append(elapsed)

                print(f"    Qiskit    | Level {level} | gates = {gates:<5} | time = {elapsed:.4f}s")

            # PennyLane
            tape = qiskit_to_pennylane_tape(qc)

            pl_opts = {
                "none": pl_none,
                "cancel": pl_cancel,
                "merge": pl_merge,
                "both": pl_both
            }

            for name, func in pl_opts.items():
                start = time.time()
                new_tape = func(tape)
                elapsed = time.time() - start
                gates = len(new_tape.operations)

                pl_gates[name].append(gates)
                pl_time[name].append(elapsed)

                print(f"    PennyLane | {name:<7} | gates = {gates:<5} | time = {elapsed:.4f}s")

        # store averages
        for level in range(4):
            avg_gates = sum(q_gates[level]) / NUM_RUNS
            avg_elapsed = sum(q_time[level]) / NUM_RUNS
            results["qiskit_gates"][level].append(avg_gates)
            results["qiskit_time"][level].append(avg_elapsed)
            print(f"  Qiskit    | Level {level} | avg gates = {avg_gates:<5.2f} | avg time = {avg_elapsed:.4f}s")

        for name in pl_gates:
            avg_gates = sum(pl_gates[name]) / NUM_RUNS
            avg_elapsed = sum(pl_time[name]) / NUM_RUNS
            results["pennylane_gates"][name].append(avg_gates)
            results["pennylane_time"][name].append(avg_elapsed)
            print(f"  PennyLane | {name:<7} | avg gates = {avg_gates:<5.2f} | avg time = {avg_elapsed:.4f}s")

    # plot outputs
    qiskit_plot_gates_vs_qubits(results)
    qiskit_plot_time_vs_qubits(results)
    pennylane_plot_gates_vs_qubits(results)
    pennylane_plot_time_vs_qubits(results)

if __name__ == "__main__":
    main()

Running Qiskit + PennyLane circuit optimization benchmarks

=== Testing n=2 qubits ===
  Trial 1/8
    Qiskit    | Level 0 | gates = 1501  | time = 0.0907s
    Qiskit    | Level 1 | gates = 977   | time = 0.0873s
    Qiskit    | Level 2 | gates = 1     | time = 0.1035s
    Qiskit    | Level 3 | gates = 1     | time = 0.0876s
    PennyLane | none    | gates = 1501  | time = 0.0238s
    PennyLane | cancel  | gates = 1339  | time = 0.0196s


C:\Users\charles\AppData\Local\Temp\ipykernel_37748\1944058523.py:4: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, _ in qc.data:


    PennyLane | both    | gates = 1163  | time = 0.0733s
    PennyLane | merge   | gates = 1344  | time = 0.0466s
  Trial 2/8
    Qiskit    | Level 0 | gates = 1491  | time = 0.0896s
    Qiskit    | Level 1 | gates = 1015  | time = 0.0950s
    Qiskit    | Level 2 | gates = 1     | time = 0.0801s
    Qiskit    | Level 3 | gates = 1     | time = 0.1000s
    PennyLane | none    | gates = 1491  | time = 0.0312s
    PennyLane | cancel  | gates = 1353  | time = 0.0179s
    PennyLane | both    | gates = 1160  | time = 0.0807s
    PennyLane | merge   | gates = 1318  | time = 0.0474s
  Trial 3/8
    Qiskit    | Level 0 | gates = 1494  | time = 0.0906s
    Qiskit    | Level 1 | gates = 1018  | time = 0.0933s
    Qiskit    | Level 2 | gates = 1     | time = 0.0858s
    Qiskit    | Level 3 | gates = 1     | time = 0.0981s
    PennyLane | none    | gates = 1494  | time = 0.0166s
    PennyLane | cancel  | gates = 1354  | time = 0.0168s
    PennyLane | both    | gates = 1179  | time = 0.0830s
    Pen

KeyboardInterrupt: 